In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import numpy as np
import torchvision
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

In [3]:
dataset = MNIST(root='data/', download=True, transform=ToTensor())

In [4]:
len(dataset)
img, label = dataset[0]
print(img.shape, label)

In [5]:
#displaying the image
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(img[0], cmap='gray') #pyplot expects the channel to be at the end so img[0] does that


In [6]:
#this function just creates training and validation data indices
# received total length of dataset and percentage of data that will be in validation set
def split_indices(n, val_pct): 
    n_val = int(val_pct * n) # 60000 * .2 = 12000
    idxs = np.random.permutation(n) # creates a permutation of n numbers i.e 60000
    return idxs[n_val:], idxs[:n_val] # training data gets [12000: end] 
                                      # validation data gets [start: 12000] 
                                      # from a permutation of n

In [7]:
train_indices, val_indices = split_indices(len(dataset), val_pct=0.2)

print(len(train_indices), len(val_indices))
print('Sample val indices: ', val_indices[:20])

In [8]:
# now we need data to be loaded according to the training and validation indicies

batch_size = 100

#creating training sampler and training dataloader
train_sampler = SubsetRandomSampler(train_indices)
train_dl = DataLoader(dataset, batch_size, sampler=train_sampler)



#creating validation sampler and validation dataloader
valid_sampler = SubsetRandomSampler(val_indices)
valid_dl = DataLoader(dataset, batch_size, sampler=valid_sampler)

# ***Creating the MODEL***

In [9]:
import torch.nn.functional as F
import torch.nn as nn

In [11]:
class MnistModel(nn.Module):
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()
        self.linear1 = nn.Linear(in_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, out_size)
        
        
    def forward(self, xb):
        #Flattening the image tensors
        xb = xb.view(xb.size(0), -1)
        out = self.linear1(xb)
        
        out = F.relu(out)
        out = self.linear2(out)
        
        return out

In [13]:
input_size = 784
num_classes = 10

model = MnistModel(input_size, hidden_size=32, out_size= num_classes) #this one uses the super constructor function


In [14]:
for t in model.parameters():
    print(t.shape)

# it is stored in transpose form row becomes columns

In [17]:
for images, labels in train_dl:
    print('images.shape ', images.shape)
    outputs = model(images) #this one is calling the forward function with a batch of data
    loss = F.cross_entropy(outputs, labels)
    print('Loss:-', loss.item())
    break

print('outputs.shape : ', outputs.shape)
print('Sample outputs :\n', outputs[:2].data)
    
    

# ****USING THE GPU****

In [18]:
torch.cuda.is_available()

In [19]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [21]:
device = get_default_device()
device

In [24]:
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x,device) for x in data]
    
    return data.to(device, non_blocking=True)

In [26]:
for images, labels in train_dl:
    print(images.shape)
    images = to_device(images, device)
    print(images.device)
    break